Imports

In [1]:
import datetime
import math
import os
import random
import sys

import gdal
import numpy as np
import scipy.misc
import imageio
import tensorflow as tf
from PIL import Image
from skimage import img_as_float
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import f1_score
%matplotlib inline
from matplotlib import pyplot as plt

/home/chc/anaconda3/envs/Segmentation/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/chc/anaconda3/envs/Segmentation/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/chc/anaconda3/envs/Segmentation/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/chc/anaconda3/envs/Segmentation

In [2]:
NUM_CLASSES = 6


class BatchColors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

# Initialization

In [3]:
list_params = ['input_path', 'output_path(for model, images, etc)', 'currentModelPath', 'trainingInstances',
                   'testing_instances', 'learningRate', 'weight_decay', 'batch_size', 'niter', 'reference_crop_size',
                   'reference_stride_crop',
                   'net_type[dilated_icpr_original|dilated_grsl|dilated_icpr_rate6|dilated_icpr_rate6_small|'
                   'dilated_icpr_rate6_densely]',
                   'distribution_type[single_fixed|multi_fixed|uniform|multinomial]', 'probValues',
                   'update_type [acc|loss]', 'process [training|validate_test|generate_final_maps]']

## Initializing individual Parameters


In [4]:
dataset = "postdam"
input_path = "/home/chc/Documents/CVML/dynamic-rs-segmentation/Postdam/"
output_path = "/home/chc/Documents/CVML/dynamic-rs-segmentation/Output/"
former_model_path = "/home/chc/Documents/CVML/dynamic-rs-segmentation/Model/"
trainingInstances = ["2_10","2_11"]
testing_instances = ["2_12"]
lr_initial = 0.01
weight_decay = 0.001
batch_size = 128
niter = 1000
reference_crop_size = 50
reference_stride_crop = 25
net_type = "dilated_icpr_original"
distribution_type = "multinomial"
values = [45,55,65,75,85]
update_type = "acc"
process = "training"
display_step = 50

### Initialize resample_batch

In [5]:
if dataset == 'vaihingen':
    resample_batch = 20
elif dataset == 'postdam':
    resample_batch = 10
else:
    print("Error! No dataset identified: ", dataset)

### Create probability distribution array 

In [6]:
def define_multinomial_probs(values, dif_prob=2):
    interval_size = values[-1] - values[0] + 1

    general_prob = 1.0 / float(interval_size)
    max_prob = general_prob * dif_prob  # for values

    probs = np.full(interval_size, (1.0 - max_prob * len(values)) / float(interval_size - len(values)))
    for i in range(len(values)):
        probs[values[i] - values[0]] = max_prob

    return probs

In [7]:
if distribution_type == 'multi_fixed':
    patch_acc_loss = np.zeros(len(values), dtype=np.float32)
    patch_occur = np.zeros(len(values), dtype=np.int32)
    patch_chosen_values = np.zeros(len(values), dtype=np.int32)

elif distribution_type == 'uniform' or distribution_type == 'multinomial':
    patch_acc_loss = np.zeros(values[-1] - values[0] + 1, dtype=np.float32)
    patch_occur = np.zeros(values[-1] - values[0] + 1, dtype=np.int32)
    patch_chosen_values = np.zeros(values[-1] - values[0] + 1, dtype=np.int32)
    probs = define_multinomial_probs(values)

### Image Loader

In [8]:
def load_images(path, instances, process, image_type='vaihingen'):
    images = []
    masks = []

    for f in instances:
        print(BatchColors.OKBLUE + 'Reading instance ' + str(f) + BatchColors.ENDC)
        
        if image_type == 'postdam':
            
            img_ndsm = img_as_float(imageio.imread(path + '1_DSM_normalisation/dsm_potsdam_0' + (
                str(f) if int(f.split("_")[1]) >= 10 else str(f.split("_")[0]) + '_0' + str(
                    f.split("_")[1])) + '_normalized_lastools.jpg'))
            
#             print(type(img_ndsm))
#             plt.imshow(img_ndsm, interpolation='nearest')
#             plt.show()
            
            if len(img_ndsm) != len(img_ndsm[0]):
                new_columns = np.zeros([len(img_ndsm), 1], dtype=type(img_ndsm[0, 0]))
                img_ndsm = np.append(img_ndsm, new_columns, axis=1)
            
            img_ndsm = np.reshape(img_ndsm, (len(img_ndsm), len(img_ndsm[0]), 1))
                        
            ds = gdal.Open(path + '4_Ortho_RGBIR/top_potsdam_' + str(f) + '_RGBIR.tif')
            img_rgb = np.empty([ds.RasterXSize, ds.RasterYSize, ds.RasterCount], dtype=np.float64)
#             print(ds.RasterCount)
#             print(img_rgb.shape)
#             plt.imshow(img_ndsm[:,:,0], interpolation='nearest')
#             plt.show()
            
            for band in range(1, ds.RasterCount + 1):
                img_rgb[:, :, band - 1] = img_as_float(np.array(ds.GetRasterBand(band).ReadAsArray()))

            if process == 'validate_test':
                img_label = imageio.imread(path + 'gts_eroded_encoding/top_potsdam_' + str(f) +
                                              '_label_noBoundary.tif')
            elif process == 'training' or process == 'crf':
                img_label = imageio.imread(path + '5_Labels_all/top_potsdam_' + str(f) + '_label.tif')
        

        full_img = np.concatenate((img_rgb, img_ndsm), axis=2)
#         print(full_img.shape)
#         plt.imshow(img_ndsm[:,:,0], interpolation='nearest')
#         plt.show()

        images.append(full_img)
        if process == 'validate_test' or process == 'training' or process == 'crf':
            masks.append(img_label)

    return np.asarray(images), np.asarray(masks)

### Load Training Data

In [9]:
print(BatchColors.WARNING + 'Reading images...' + BatchColors.ENDC)
training_data, training_labels = load_images(input_path, trainingInstances, process, image_type=dataset)
print("Training Data Loaded ...")

Reading images...
Reading instance 2_10
Reading instance 2_11


### Load Testing Data

In [10]:
testing_data, testing_labels = load_images(input_path, testing_instances, process, image_type=dataset)
print("Testing Data Loaded ...")

Reading instance 2_12
Testing Data Loaded ...


### Create Training Class Distribution

In [11]:
def create_distributions_over_classes(labels, crop_size, stride_crop):
    classes = [[[] for i in range(0)] for i in range(NUM_CLASSES)]
    print(len(classes))
    print(len(labels))
    print("------------------")
    for k in range(len(labels)):
        w, h, channels = labels[k].shape

        for i in range(0, w, stride_crop):
            for j in range(0, h, stride_crop):
                cur_map = k
                cur_x = i
                cur_y = j
                patch_class = labels[cur_map][cur_x:cur_x + crop_size, cur_y:cur_y + crop_size]

                if len(patch_class) != crop_size and len(patch_class[0]) != crop_size:
                    cur_x = cur_x - (crop_size - len(patch_class))
                    cur_y = cur_y - (crop_size - len(patch_class[0]))
                    patch_class = labels[cur_map][cur_x:cur_x + crop_size, cur_y:cur_y + crop_size]
                elif len(patch_class) != crop_size:
                    cur_x = cur_x - (crop_size - len(patch_class))
                    patch_class = labels[cur_map][cur_x:cur_x + crop_size, cur_y:cur_y + crop_size]
                elif len(patch_class[0]) != crop_size:
                    cur_y = cur_y - (crop_size - len(patch_class[0]))
                    patch_class = labels[cur_map][cur_x:cur_x + crop_size, cur_y:cur_y + crop_size]

                if patch_class.shape == (crop_size, crop_size, channels):
                    count = np.bincount(patch_class.astype(int).flatten())
                    # print(count)
                    # print(len(count))
                    # print(int(np.argmax(count)))
                    classes[int(np.argmax(count))].append((cur_map, cur_x, cur_y))
                else:
                    print(BatchColors.FAIL + "Error create_distributions_over_classes: Current patch size is " + str(
                        len(patch_class)) + "x" + str(len(patch_class[0])) + BatchColors.ENDC)
                    return

    for i in range(len(classes)):
        print(BatchColors.OKBLUE + 'Class ' + str(i + 1) + ' has length ' + str(len(classes[i])) + BatchColors.ENDC)

    return classes

In [12]:
print(BatchColors.WARNING + 'Creating TRAINING class distribution...' + BatchColors.ENDC)
training_class_distribution = create_distributions_over_classes(training_labels, crop_size=reference_crop_size,
                                                                        stride_crop=reference_stride_crop)

Creating TRAINING class distribution...
6
2
------------------


IndexError: list index out of range